##### Copyright 2024 Google LLC.

In [ ]:
# @title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# CodeGemma - Common use cases
This notebook demonstrates the basic task that Gemma can solve by using the right prompting.
<table align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/google-gemini/gemma-cookbook/blob/main/CodeGemma/%5BCodeGemma_1%5DCommon_use_cases.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
</table>

## Setup

### Select the Colab runtime
To complete this tutorial, you'll need to have a Colab runtime with sufficient resources to run the Gemma model. In this case, you can use a T4 GPU:

1. In the upper-right of the Colab window, select **▾ (Additional connection options)**.
2. Select **Change runtime type**.
3. Under **Hardware accelerator**, select **T4 GPU**.

### Gemma setup

To complete this tutorial, you'll first need to complete the setup instructions at [Gemma setup](https://ai.google.dev/gemma/docs/setup). The Gemma setup instructions show you how to do the following:

* Get access to Gemma on kaggle.com.
* Select a Colab runtime with sufficient resources to run
  the Gemma 2B model.
* Generate and configure a Kaggle username and an API key as Colab secrets.

After you've completed the Gemma setup, move on to the next section, where you'll set environment variables for your Colab environment.


### Configure your credentials

Add your your Kaggle credentials to the Colab Secrets manager to securely store it.

1. Open your Google Colab notebook and click on the 🔑 Secrets tab in the left panel. <img src="https://storage.googleapis.com/generativeai-downloads/images/secrets.jpg" alt="The Secrets tab is found on the left panel." width=50%>
2. Create new secrets: `KAGGLE_USERNAME` and `KAGGLE_KEY`
3. Copy/paste your username into `KAGGLE_USERNAME`
3. Copy/paste your key into `KAGGLE_KEY`
4. Toggle the buttons on the left to allow notebook access to the secrets.


In [ ]:
import os
from google.colab import userdata

# Note: `userdata.get` is a Colab API. If you're not using Colab, set the env
# vars as appropriate for your system.
os.environ["KAGGLE_USERNAME"] = userdata.get("KAGGLE_USERNAME")
os.environ["KAGGLE_KEY"] = userdata.get("KAGGLE_KEY")

### Install dependencies
Run the cell below to install all the required dependencies.

In [ ]:
!pip install -q -U keras keras-nlp

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 570.5/570.5 kB 15.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.2/311.2 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 950.8/950.8 kB 14.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 17.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 589.8/589.8 MB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.3/5.3 MB 81.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 95.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 85.5 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tf-keras 2.15.1 requires tensorflow<2.16,>=2.15, but you have tensorflow 2.16.1 which is incompatible

## Exploring prompting capabilities

### CodeGemma

CodeGemma models are text-to-text and text-to-code decoder-only models that specializes in code completion and code generation tasks. The CodeGemma 2B and 7B variants are specially tuned for code infilling tasks.

This example uses CodeGemma's fill-in-the-middle (FIM) capability to complete code based on the surrounding context. This is particularly useful in code editor applications for inserting code where the text cursor is based on the code around it (before and after the cursor).

CodeGemma lets you use 4 user-defined tokens - 3 (`<|fim_prefix|>`, `<|fim_suffix|>`, `<|fim_middle|>`) for FIM and a `<|file_separator|>` token for multi-file context support. Use these to define constants. More about that in the next cells.


In [ ]:
import os
import keras
import keras_nlp
from google.colab import userdata

keras.config.set_floatx("bfloat16")
os.environ["KAGGLE_USERNAME"] = userdata.get("KAGGLE_USERNAME")
os.environ["KAGGLE_KEY"] = userdata.get("KAGGLE_KEY")

In [ ]:
# Load CodeGemma
codegemma = keras_nlp.models.GemmaCausalLM.from_preset("code_gemma_1.1_2b_en")

100%|██████████| 143/143 [00:00<00:00, 126kB/s]


100%|██████████| 501/501 [00:00<00:00, 428kB/s]


100%|██████████| 4.67G/4.67G [02:25<00:00, 34.4MB/s]


100%|██████████| 315/315 [00:00<00:00, 413kB/s]


100%|██████████| 4.04M/4.04M [00:00<00:00, 7.16MB/s]


In [ ]:
# Tokens
BEFORE_CURSOR = "<|fim_prefix|>"
AFTER_CURSOR = "<|fim_suffix|>"
AT_CURSOR = "<|fim_middle|>"
FILE_SEPARATOR = "<|file_separator|>"
END_TOKEN = codegemma.preprocessor.tokenizer.end_token
stop_tokens = (BEFORE_CURSOR, AFTER_CURSOR, AT_CURSOR, FILE_SEPARATOR, END_TOKEN)

In [ ]:
stop_token_ids = tuple(
    codegemma.preprocessor.tokenizer.token_to_id(x) for x in stop_tokens
)

#### Prompting example: Code infilling

In [ ]:
import re


# Helpers
def split_response_by_token(response):
    mapping = {}
    parts = re.split(r"(<\|[^\|\>]+\|\>)", response)
    parts = [item for item in parts if len(item)]
    for token in stop_tokens[:3]:
        mapping[token] = ""

        try:
            idx = parts.index(token)
            if parts[idx + 1] not in stop_tokens:
                mapping[token] = parts[idx + 1]
        except (ValueError, IndexError):
            pass

    return mapping

In [ ]:
prefix = "def calculate_area_of_rectangle(a: int, b: int) -> int:"
suffix = "\n    return area"
prompt = f"<|fim_prefix|>{prefix}<|fim_suffix|>{suffix}<|fim_middle|>"

response = codegemma.generate(prompt, stop_token_ids=stop_token_ids)
parts = split_response_by_token(response)

print("--- RAW Response ---")
print(response)

print("\n--- The generated (FIM) piece of code: ---")
print(parts[AT_CURSOR])

print("\n--- The whole function: ---")
print(parts[BEFORE_CURSOR], parts[AT_CURSOR], parts[AFTER_CURSOR])

--- RAW Response ---
<|fim_prefix|>def calculate_area_of_rectangle(a: int, b: int) -> int:<|fim_suffix|>
    return area<|fim_middle|>
    """
    Calculate the area of a rectangle.

    Args:
        a (int): The length of the rectangle.
        b (int): The width of the rectangle.

    Returns:
        int: The area of the rectangle.
    """
    area = a * b<|file_separator|>

--- The generated (FIM) piece of code: ---

    """
    Calculate the area of a rectangle.

    Args:
        a (int): The length of the rectangle.
        b (int): The width of the rectangle.

    Returns:
        int: The area of the rectangle.
    """
    area = a * b

--- The whole function: ---
def calculate_area_of_rectangle(a: int, b: int) -> int: 
    """
    Calculate the area of a rectangle.

    Args:
        a (int): The length of the rectangle.
        b (int): The width of the rectangle.

    Returns:
        int: The area of the rectangle.
    """
    area = a * b 
    return area


#### Prompting example: Code Generation
_Note: While the 2B version of CodeGemma was primarily designed for code completion, it can also handle basic code generation tasks. However, for optimal code generation results, we recommend using the instruction-tuned 7B model._

In [ ]:
prompt = """Write a one-liner in Python that check if a year is a leap year.
Examples:
>>> is_a_leap_year(2016)
True
>>> is_a_leap_year(2001)
False
>>> is_a_leap_year(2052)
True
def is_a_leap_year(year: int) -> bool:"""

response = codegemma.generate(prompt, max_length=128)
print(response)

Write a one-liner in Python that check if a year is a leap year.
Examples:
>>> is_a_leap_year(2016)
True
>>> is_a_leap_year(2001)
False
>>> is_a_leap_year(2052)
True
def is_a_leap_year(year: int) -> bool:
    return year % 4 == 0 and (year % 100 != 0 or year % 400 == 0)
<|file_separator|>


In [ ]:
# We need to restart the session to erase GPU RAM memory
# and load a new model
get_ipython().kernel.do_shutdown(True)

{'status': 'ok', 'restart': True}